In [ ]:
import time
import random
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd

import selenium
import webdriver_manager
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException

from crawler import get_url
from test import QuantusQA
from settings import filter_conditions

from database import AccessDataBase
db = AccessDataBase("quantus_test")
qqa = QuantusQA()

In [ ]:
wd = qqa.init_page(**filter_conditions)

In [ ]:
filter_conditions["removed_sectors"] = []

In [ ]:
qqa.test_back_port(wd, kind=0, **filter_conditions)

In [ ]:
# # 이미 설정이 완료되었을 때
# # 과거 거래내역 추출
# wd.get("https://v2.quantus.kr/port/port/past")
# time.sleep(1.5)
# backtest(wd)

In [ ]:
# # 이미 설정이 완료되었을 때
# # 10분위 테스트
# wd.get("https://v2.quantus.kr/backtest/backtest/decile")
# time.sleep(1.5)
# backtest(wd)

In [ ]:
# reset(wd)

In [ ]:
wd.quit()

---
### Test: 0

In [ ]:
wd.quit()

In [ ]:
variables = {}

In [ ]:
url = "https://v2.quantus.kr/backtest/universe"
# url = "v2.quantus.kr/backtest/universe"

wd = get_url(url, window=True, image=True, logging=True)

In [ ]:
wd.get_log(log_type="browser")

In [ ]:
def select_level(wd, level: int = None):
    
    d = {
        0: "초급",
        1: "중급",
        2: "고급",
    }
    if not level:
        keys = list(d.keys())
        level = random.randint(min(keys), max(keys))

    variables["level"] = d[level]
    # select level
    levels = wd.find_elements(By.CLASS_NAME, "css-2ho4nc")
    levels[level].click()

    # apply
    wd.find_element(By.CLASS_NAME, "yes").click()
    wd.find_element(By.CLASS_NAME, "css-1lcj2w").click()

In [ ]:
select_level(wd, level=2)

In [ ]:
def _login(wd, login: int = None):
    wd.get("https://v2.quantus.kr/login")
    d = {
        0: "구글",
        1: "카카오",
    }

    if not login:
        keys = list(d.keys())
        login = random.randint(min(keys), max(keys))


    login = d[login]
    variables["login"] = login

    wd.find_element(By.ID, login).click()

    # Enter the login info
    email = 'wjddustjsla@naver.com'
    pw = 'jys1013011!'

    elm_email = wd.find_element(By.ID, "input-loginKey")
    elm_pw = wd.find_element(By.ID, "input-password")

    # input login info
    elm_email.send_keys(email)
    elm_pw.send_keys(pw)

    # click login button
    # wd.find_element(By.CLASS_NAME, "btn_g highlight").click()
    wd.find_element(By.XPATH, "/html/body/div/div/div/main/article/div/div/form/div[4]/button[1]").click()

In [ ]:
_login(wd, login=0)

In [ ]:
# strategy name

if not strategy_name:
    now = datetime.now().strftime("%y%m%d_%H_%M_%S")
    strategy_name = f"test_{now}"
print(f"Strategy name: {strategy_name}")

# Enter the strategy name
strategy_name_xpath = "/html/body/div[1]/div/div[2]/div[1]/div/div[1]/input"
elm_strategy_name = wd.find_element(By.XPATH, strategy_name_xpath)
elm_strategy_name.send_keys(strategy_name)
variables["strategy_name"] = strategy_name

In [ ]:
# 유티버스 선택

d = {
    0: "한국",
    1: "미국",
    2: "KOSPI",
    3: "KOSDAQ",
}
if not universe:
    keys = list(d.keys())
    universe = random.randint(min(keys), max(keys))
    variables["universe"] = d[universe]

print(f"Universe: {d[universe]}")

wd.find_element(By.CLASS_NAME, "css-cy3vpx").click()
universes = wd.find_elements(By.CLASS_NAME, "css-zf30n6")

universe = universes[universe]
universe.click()

In [ ]:
wd.refresh()
variables = {}

In [ ]:

def scraping_sectors():
    # 기본 필터, 제외할 섹터 스크레이핑
    wd = get_url("https://v2.quantus.kr/backtest/universe")
    soup = BeautifulSoup(wd.page_source, "lxml")
    filter_sector_class = "css-192m3r6"

    filters = soup.find("div", "css-i2b1or").find_all("div", filter_sector_class)
    sectors = soup.find("div", "css-16ww4h3").find_all("div", filter_sector_class)

    d = {
        "기본 필터": {},
        "제외할 섹터": {},
    }
    for i in range(len(filters) + len(sectors)):
        if i < len(filters):
            d["기본 필터"][i] = filters[i].text.strip()
            
        else:
            d["제외할 섹터"][i] = sectors[i - len(filters)].text.strip()
            
    return d

In [ ]:
removed_filters, removed_sectors = None, None

soup = BeautifulSoup(wd.page_source, "lxml")
filter_sector_class = "css-192m3r6"

filters = soup.find("div", "css-i2b1or").find_all("div", filter_sector_class)
sectors = soup.find("div", "css-16ww4h3").find_all("div", filter_sector_class)
len_filters = len(filters)

# 기본 필터 & 제외할 섹터 elements
removed_elms = wd.find_elements(By.CLASS_NAME, filter_sector_class)

if not removed_filters:
    removed_filters = random.sample(filters, random.randint(0, len(filters)))
variables["removed_filters"] = []
for filter in removed_filters:
    variables["removed_filters"].append(filter.text.strip())
    i = filters.index(filter)
    removed_elms[i].click()

if not removed_sectors:
    removed_sectors = random.sample(sectors, random.randint(0, len(sectors)))
variables["removed_sectors"] = []
for sector in removed_sectors:
    variables["removed_sectors"].append(sector.text.strip())
    i = sectors.index(sector) + len_filters
    removed_elms[i].click()


# # check filter
# i = 0
# variables["removed_filter"] = []
# for filter in filters:
#     c = round(random.random(), 0)
#     if c == 1:
#         elm_filter_sector[i].click()
#         filter = filter.text
#         variables["removed_filter"].append(filter)
#     i += 1
    
# time.sleep(3)

# # check sector 
# variables["removed_sector"] = []
# for sector in sectors:
#     c = round(random.random(), 0)
#     if c == 1:
#         elm_filter_sector[i].click()
#         sector = sector.text
#         variables["removed_sector"].append(sector)
#     i += 1

In [ ]:
variables

In [ ]:
# click '다음' button
wd.find_element(By.CLASS_NAME, "css-etpc0o").click()

In [ ]:
# factor 선택

soup = BeautifulSoup(wd.page_source, "lxml")

factor_area_class = "css-kwx3if"
factor_kind_class = "css-12hmsi2"
factor_elm_class = "css-192m3r6"

factor_area = soup.find_all("div", factor_area_class)
elm_factors = wd.find_elements(By.CLASS_NAME, factor_elm_class)

i = 0
for fs in factor_area:
    factor_kind = fs.find("div", factor_kind_class).text.strip()
    variables[factor_kind] = []
    fs_nms = fs.find_all("div", factor_elm_class)
    for fs_nm in fs_nms:
        c = round(random.random(), 0)
        if c == 1:
            fs_nm = fs_nm.text.strip().replace("?", "")
            variables[factor_kind].append(fs_nm)
            elm_factors[i].click()
            time.sleep(0.5)
        i += 1

In [ ]:
# click '다음' button
wd.find_element(By.CLASS_NAME, "css-etpc0o").click()

In [ ]:
# consts.py

init_amounts = 2000 # 만원 단위
fees = 1 # %(퍼센트)
stock_counts = 10
stop_loss = 26 # %(퍼센트)
profit = 26 # %(퍼센트)

rebalancing_period, rebalancing_method, rebalancing_strategy = None, None, None

backtest_conditions_input = {
    0: ["초기 투자 금액", init_amounts],
    1: ["거래 수수료", fees],
    2: ["종목 수", stock_counts],
    3: ["손절기준", stop_loss],
    4: ["익절기준", profit],
    5: ["리벨런싱 기간", rebalancing_period]
}

In [ ]:
all_elms_class = "css-cy3vpx" # input box & select box

''' input box
0. 초기 투자 금액 *
1. 거래 수수료 *
2. 종목수 * (20으로 입력되어 있음)
3. 손절기준 (입력안할 시 비활성화)
4. 익절기준 (입력안할 시 비활성화)
'''
input_class = "css-1tmhs6j"

''' select box
5. 리밸런싱 기간 *
6. 비중 조절 방법 * ('동일 비중'으로 지정되어 있음)
7. 리밸런싱 전략 * ('분할 매수/매도'로 지정되어 있음)
'''
select_class = "css-cy3vpx"


In [ ]:
wd.refresh()

In [ ]:
all_elms = wd.find_elements(By.CLASS_NAME, all_elms_class)
input_elms = wd.find_elements(By.CLASS_NAME, input_class)
select_elms = wd.find_elements(By.CLASS_NAME, select_class)

soup = BeautifulSoup(wd.page_source, "lxml")
all_boxes = soup.find_all("div", all_elms_class)

i = 0
for elm in input_elms:
    cond, val = backtest_conditions_input[i]
    variables[cond] = val
    elm.clear()
    time.sleep(1.5)
    elm.send_keys(val)
    if i == 1:
        # 리밸런싱 기간 선택
        pass
    i += 1

In [ ]:
# 리밸런싱 기간 선택
wd.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[3]/div/div[6]/div[1]/div[2]/input").click()

d = {
    1: "월별",
    2: "분기별",
    3: "반기별",
    4: "매년",
}

# keys = list(d.keys())
# j = random.randint(min(keys), max(keys))

variables["rebalancing_period"] = d[rebalancing_period]
print(d[rebalancing_period])
rebalancing_period_xpath = f"/html/body/div[1]/div/div[2]/div[3]/div/div[6]/div[3]/div[{rebalancing_period}]/div/input"
wd.find_element(By.XPATH, rebalancing_period_xpath).click()
time.sleep(1)

In [ ]:
# save strategy
save_button_class = "css-1o923v9"
wd.find_element(By.CLASS_NAME, save_button_class).click()

check_alert(wd)

In [ ]:
# backtest
backtest_button_class = "css-etpc0o"
wd.find_element(By.CLASS_NAME, backtest_button_class).click()

In [ ]:
wd.get_log(log_type="browser")

In [ ]:
from selenium.common.exceptions import NoSuchElementException

def check_alert(wd):
    alert_classes = [
        "css-1x7ny0l",
        "css-1aq7lc0",
        "css-1t6s947",
    ]
    check_classes = [
        "css-1lcj2w",
        "css-73nldo",
        "css-1lcj2w",
        "css-28mzi7"
    ]

    soup = BeautifulSoup(wd.page_source, "lxml")
    alert_msg = None
    i = 0
    ck = False
    while i < len(alert_classes):
        alert_area = soup.find("div", alert_classes[i])
        if alert_area:
            alert_msg = alert_area.text
            j = 0
            while j < len(check_classes):
                check_class = check_classes[j]
                check_btn = alert_area.find("div", check_class)
                if check_btn:
                    wd.find_element(By.CLASS_NAME, check_class).click()
                    ck = True
                    break
                j += 1
            
            if ck:
                break
        i += 1
    
    return alert_msg
        
    
    
    # if soup.find("div", alert_class_0):
    #     alert_msg = soup.find("div", alert_class_0).text[:-2]
    #     wd.find_element(By.CLASS_NAME, alert_check_class_0).click()
    # elif soup.find("div", alert_class_1):
    #     try:
    #         alert_msg = soup.find("div", alert_class_1).text[:-4]
    #         wd.find_element(By.CLASS_NAME, alert_check_class_1).click()
    #     except NoSuchElementException:
    #         alert_msg = soup.find("div", alert_class_1).text[:-2]
    #         wd.find_element(By.CLASS_NAME, alert_check_class_2).click()
    # else:
    #     alert_msg = None 
    
    # return alert_msg

In [ ]:
check_alert(wd)

In [ ]:
# go to strategy page
wd.get("https://v2.quantus.kr/mypage/strategy")

# 가장 최근 전략 적용
wd.find_element(By.CLASS_NAME, "orange_button").click()

# alert box 처리
variables["alert_msg"] = []
variables["alert_msg"].append(check_alert(wd))
variables["alert_msg"].append(check_alert(wd))

In [ ]:
# wd.get("https://v2.quantus.kr/backtest/backtest")
wd.find_element(By.CLASS_NAME, "css-o80j1r").click()

In [ ]:
# click '다음' button
wd.find_element(By.CLASS_NAME, "css-etpc0o").click()

In [ ]:
# backtest
backtest_button_class = "css-etpc0o"
wd.find_element(By.CLASS_NAME, backtest_button_class).click()

In [ ]:
wd.quit()

In [ ]:
# click '포트폴리오 추출' 메뉴

wd.find_element(By.CLASS_NAME, "css-dgyndo").click()

---
#### Test: logging

In [ ]:
import os
import re
import sys
import time
import json
import socket
import requests
# import pickle
import numpy as np
import pandas as pd

# Scrapping
from bs4 import BeautifulSoup
from seleniumwire import webdriver
from user_agent import generate_user_agent
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.alert import Alert
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

def get_url_log(url, window=False, image=False, limit=1):
    ''' Set up webdriver, useragent & Get url '''
    
    wd = None
    socket.setdefaulttimeout(30)
    # url parsing 시도횟수
    attempts = 0 
    # limit회 이상 parsing 실패시 pass
    while attempts < limit:
        # try:  
        attempts += 1
        # user agent
        options = Options() 
        userAgent = generate_user_agent(os=('mac', 'linux'), navigator='chrome', device_type='desktop')
        options.add_argument('window-size=1920x1080')
        options.add_argument("--disable-gpu")
        options.add_argument('--disable-extensions')
        options.add_argument(f'user-agent={userAgent}')
        options.add_argument("--start-fullscreen")
        
        if not window:
            options.add_argument('headless')
        if not image:
            options.add_argument('--blink-settings=imagesEnabled=false')

        dc = DesiredCapabilities.CHROME
        #log 종류는 OFF, SEVERE, WARNING, INFO, DEBUG, ALL 존재
        dc['goog:loggingPrefs'] = {'browser': 'ALL'}
        # web driver 
        wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options, desired_capabilities=dc)
        wd.get(url)
        wd.implicitly_wait(5)
        break

        # # 예외처리
        # except Exception as e:
        #     print(f'\n\nError: {str(e)}\n\n')
            
        #     # time.sleep(60)
        #     try:
        #         wd.quit()
        #     except:
        #         pass
        #     wd = None
    return wd

In [ ]:
wd = get_url_log(url, True, True)

In [ ]:
wd.get_log(log_type="browser")

In [ ]:
_requests = wd.requests

In [ ]:
i = 0
for r in _requests:
    res =  _requests[i].response
    if res is None:
        pass
    elif res.status_code == 404:
        break
    i += 1

In [ ]:

def scraping_factors():
    # factor scraping
    
    wd = get_url("https://v2.quantus.kr/backtest/factors")
    soup = BeautifulSoup(wd.page_source, "lxml")

    factor_area_class = "css-kwx3if"
    factor_kind_class = "css-12hmsi2"
    factor_elm_class = "css-1z2md3"
    available_factor_class = "css-192m3r6"

    factor_area = soup.find_all("div", factor_area_class)
    elm_factors = wd.find_elements(By.CLASS_NAME, factor_elm_class)

    variables = {}
    i = 0
    for fs in factor_area:
        factor_kind = fs.find("div", factor_kind_class).text.strip()
        variables[factor_kind] = {}
        variables[factor_kind]["unavailable"] = []
        fs_nms = fs.find_all("div", factor_elm_class)
        for fs_nm in fs_nms:
            _fs_nm = fs_nm.text.strip().replace("?", "")
            if fs_nm.find("div", available_factor_class):
                variables[factor_kind][i] = _fs_nm
                i += 1
            else:
                variables[factor_kind]["unavailable"].append(_fs_nm)

    return variables

In [ ]:
variables

---
### Test: 백테스트, 포트추출, 10분위 테스트

In [ ]:
from test import *
from settings import filter_conditions
filter_conditions

In [ ]:
def test_back_port(wd, kind: int = None, **kwargs):
    
    if not kind:
        kind = random.randint(0, 1)
    elif kind == 0:
        # backtest
        wd.get("https://v2.quantus.kr/backtest/universe")
        kind = "backtest"
    elif kind == 1:
        # port 
        wd.get("https://v2.quantus.kr/port/universe")
        kind = "port"
    else:
        raise TypeError("올바른 kind(0: backtest, 1: port)를 입력하세요")
    
    print(kind)
    # 유니버스 선택
    select_universe(wd, kwargs["strategy_name"], kwargs["universe"], kwargs["removed_filters"], kwargs["removed_sectors"])
    time.sleep(2)
    click_next_btn(wd)

    # 팩터 선택
    select_factors(wd, kwargs["factors"])
    time.sleep(2)
    click_next_btn(wd)

    set_conditions(wd, **kwargs["conditions"])
        
    # 전략 저장
    save_strategy(wd)
    check_alert(wd)
    time.sleep(1.5)
    
    # 백테스트 or 포트추출 실행
    backtest(wd)
    
    variables['logs'] = wd.get_log(log_type="browser")
    return variables

In [ ]:
wd.quit()

In [ ]:
time.sleep(1.5)
url = "https://v2.quantus.kr/"
wd = get_url(url, window=True, image=True, logging=True)
time.sleep(3.5)

# 로그인
sign_in(wd, filter_conditions["login"], filter_conditions["email"], filter_conditions["pw"])

In [ ]:
kinds = [0, 1, 0, 1]
kinds = [0]
variables_list = []
for kind in kinds:
    variables = test_back_port(wd, kind, **filter_conditions)
    variables_list.append(variables)
    reset(wd)

In [ ]:
variables

In [ ]:
reset(wd)
variables = test_back_port(wd, kind=0, **filter_conditions)

In [ ]:
wd.current_url

In [ ]:
# 유니버스 선택
select_universe(wd)
time.sleep(2)
click_next_btn(wd)

# 팩터 선택
select_factors(wd)
time.sleep(2)
click_next_btn(wd)

# set_conditions(wd, **conditions)

In [ ]:
# 전략 저장
save_strategy(wd)
check_alert(wd)

# 백테스트 or 포트추출 실행
backtest(wd)

In [ ]:
variables['logs'] = wd.get_log(log_type="browser")
variables_list.append(variables)

In [ ]:
print(wd.get_log(log_type="browser"))
# check_alert(wd)

In [ ]:
reset(wd)

In [ ]:
wd.refresh()

In [ ]:
wd.quit()

---
#### get data

In [ ]:
def read_result(kind):
    for p in sys.path:
        if "Users" in p: 
            user_name = p.split("/")[2]    
            download_dir = f"/Users/{user_name}/Downloads"
            break

    files = os.listdir(download_dir)
    files = [os.path.join(download_dir, f) for f in files]
    
    # 생성일자 기준 정렬
    files.sort(key=lambda x: os.path.getmtime(x))

    if kind == 0:
        # backtest
        path = [f for f in files if f.endswith(".html")][-1]
        
        _wd = get_url("file://" + path)
        soup = BeautifulSoup(_wd.page_source, "lxml")
        _wd.quit()
        
        tbl = soup.find("div", {"id": "right"}).find("table")
        tbl_trs = tbl.find_all("tr")

        results = {}
        # Cumulative Return
        tbl_tr_td = tbl_trs[4].find_all("td")
        results[tbl_tr_td[0].text] = tbl_tr_td[1].text
        # CAGR
        tbl_tr_td = tbl_trs[5].find_all("td")
        results[tbl_tr_td[0].text] = tbl_tr_td[1].text
        # Max Drawdown	
        tbl_tr_td = tbl_trs[15].find_all("td")
        results[tbl_tr_td[0].text] = tbl_tr_td[1].text
        
    elif kind == 1:
        # port extraction
        path = [f for f in files if f.endswith(".csv")][-1]
        df = pd.read_csv(path)
        print(df)
        results = df["Name"].tolist()
        
    return results


In [ ]:
for p in sys.path:
    if "Users" in p:
        user_name = p.split("/")[2]
        download_dir = f"/Users/{user_name}/Downloads"
        break
    
download_dir

In [ ]:
files = os.listdir(download_dir)
files = [os.path.join(download_dir, f) for f in files]

In [ ]:
files.sort(key=lambda x: os.path.getmtime(x))
files

In [ ]:
wd.quit()

In [ ]:
path = read_result(0)


_wd = get_url("file://" + path)
soup = BeautifulSoup(_wd.page_source, "lxml")
_wd.quit()

tbl = soup.find("div", {"id": "right"}).find("table")
tbl_trs = tbl.find_all("tr")

results = {}

# Cumulative Return
tbl_tr_td = tbl_trs[4].find_all("td")
results[tbl_tr_td[0].text] = tbl_tr_td[1].text

# CAGR
tbl_tr_td = tbl_trs[5].find_all("td")
results[tbl_tr_td[0].text] = tbl_tr_td[1].text

# Max Drawdown	
tbl_tr_td = tbl_trs[15].find_all("td")
results[tbl_tr_td[0].text] = tbl_tr_td[1].text

results

In [ ]:
variables = {}

df = pd.read_csv("/Users/yeonseojung/Downloads/test_230109_17_44_55_quantile_result.csv")
data = df.iloc[4, :].tolist()
variables[data[0].strip() + "s"] = data[1:]

data = df.iloc[5, :].tolist()
variables[data[0].strip() + "s"] = data[1:]

data = df.iloc[10, :].tolist()
variables[data[0].strip() + "s"] = data[1:]

variables

In [ ]:
df = pd.read_csv("/Users/yeonseojung/Downloads/퀀터스 전략_portfolio_result.csv")
df

---
### Test: factors

In [ ]:
factors = [1, 2, 63]

In [ ]:
variables = {}

factor_area_class = "css-kwx3if"
factor_kind_class = "css-12hmsi2"
factor_elm_class = "css-192m3r6"

soup = BeautifulSoup(wd.page_source, "lxml")
factor_area = soup.find_all("div", factor_area_class)
elm_factors = wd.find_elements(By.CLASS_NAME, factor_elm_class)
factor_nms = soup.find_all("div", factor_elm_class)

factor_kinds = {}
for fs in factor_area:
    factor_kind = fs.find("div", factor_kind_class).text.strip()
    variables[factor_kind] = []
    nms = fs.find_all("div", factor_elm_class)
    factor_kinds[factor_kind] = [nm.text.replace("?", "").strip() for nm in nms]
    
# if len(factors) == 0:
#     factors = random.sample(range(len(elm_factors)), random.randint(0, len(elm_factors)))
    
    
# for i in factors:
#     elm_factor = elm_factors[i]
#     elm_factor.click()
#     time.sleep(0.25)
#     factor = factor_nms[i].text.replace("?", "").strip()
    
#     for k, v in factor_kinds.items():
#         if factor in v:
#             variables[k].append(factor)
#             break

In [ ]:
variables

In [ ]:
factor_kinds

---
### Test: 마이페이지

In [ ]:
# wd.get("https://v2.quantus.kr/mypage")

In [ ]:
# '마이페이지'
wd.find_element(By.CLASS_NAME, "css-1r42moo").click()

In [ ]:
# '전략조회'
# wd.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div/div/div[1]/div[2]/button[2]").click()
wd.get("https://v2.quantus.kr/mypage/strategy")

In [ ]:
# 가장 최근에 저장한 전략 적용
wd.find_element(By.CLASS_NAME, "orange_button").click()
check_alert(wd)
check_alert(wd)

In [ ]:
def remove_strategy(wd):
    wd.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div/div/div[1]/div/div[2]/div/div[2]/div[1]/div[2]/div[3]/img").click()
    check_alert(wd)
    check_alert(wd)

def remove_startegies(wd):
    # 가장 최근에 저장한 전략 삭제
    i = 0
    while True:
        try:
            remove_strategy(wd)
        except:
            break
        i += 1
        
    print(f"삭제한 전략 수: {i}")

---
### Test

In [1]:
import time
import ast
import traceback
import random
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd

import selenium
import webdriver_manager
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException

from crawler import get_url, click_elm
from test import QuantusQA
from settings import filter_conditions

from database import AccessDataBase
db = AccessDataBase("quantus_test")
qqa = QuantusQA()

def testing(wd, filter_conditions):
    qqa.__init__()
    
    # 초기 설정 (유니버스, 필터, 팩터)
    # backtest
    qqa.test_back_port(wd, kind=0, **filter_conditions)

    # 이미 설정이 완료되었을 때
    # 10분위 테스트
    # wd.get("https://v2.quantus.kr/backtest/backtest/decile")
    click_elm(wd, "id", "10분위 테스트")
    time.sleep(1.5)
    qqa.get_result(wd, kind=2)
    
    # 이미 설정이 완료되었을 때
    # 포트 추출
    # wd.get("https://v2.quantus.kr/port/port")
    # click '포트폴리오 추출', go to '트레이딩 설정'
    elms = wd.find_elements(By.CLASS_NAME, "css-o80j1r")
    elms[1].click()
    time.sleep(1.5)
    qqa.click_next_btn(wd)
    qqa.click_next_btn(wd)
    qqa.get_result(wd, kind=1)
    
    # gauges
    qqa.variables["gauges"] = qqa.gauges

In [2]:
# init page
wd = qqa.init_page(**filter_conditions)

In [3]:
# delete remove strategy
qqa.remove_startegies(wd)

ALERT: 전략 삭제"test_230113_18_40_07" 전략을 삭제하시겠습니까?
ALERT: 전략 삭제 완료"test_230113_18_40_07" 전략 삭제가 완료되었습니다.
ALERT: 전략 삭제"test_230113_18_28_40" 전략을 삭제하시겠습니까?
ALERT: 전략 삭제 완료"test_230113_18_28_40" 전략 삭제가 완료되었습니다.
ALERT: 전략 삭제"test_230113_16_35_04" 전략을 삭제하시겠습니까?
ALERT: 전략 삭제 완료"test_230113_16_35_04" 전략 삭제가 완료되었습니다.
ALERT: 전략 삭제"test_230113_16_27_55" 전략을 삭제하시겠습니까?
ALERT: 전략 삭제 완료"test_230113_16_27_55" 전략 삭제가 완료되었습니다.
ALERT: 전략 삭제"test_230113_16_24_45" 전략을 삭제하시겠습니까?
ALERT: 전략 삭제 완료"test_230113_16_24_45" 전략 삭제가 완료되었습니다.
ALERT: 전략 삭제"test_230113_16_17_36" 전략을 삭제하시겠습니까?
ALERT: 전략 삭제 완료"test_230113_16_17_36" 전략 삭제가 완료되었습니다.
ALERT: 전략 삭제"test_230113_16_08_56" 전략을 삭제하시겠습니까?
ALERT: 전략 삭제 완료"test_230113_16_08_56" 전략 삭제가 완료되었습니다.
ALERT: 전략 삭제"test_230113_16_01_33" 전략을 삭제하시겠습니까?
ALERT: 전략 삭제 완료"test_230113_16_01_33" 전략 삭제가 완료되었습니다.


In [ ]:
# filter_conditions["conditions"]["stop_loss"] = 0
# filter_conditions["conditions"]["profit"] = 0

# filter_conditions["factors"] = [0, 1, 3, 4, 8, 24, 54, 55, 56, 57, 60, 61]
# filter_conditions

In [ ]:
# filter_conditions["custom_factors"]["denominators"] = [1, 2]
# filter_conditions["custom_factors"]["numerators"] = [2, 1]

# filter_conditions

In [ ]:
qqa.__init__()
qqa.variables

     
#### 0. 단일 테스트

In [ ]:
filter_conditions["custom_factors"]["denominators"] = [1]
filter_conditions["custom_factors"]["numerators"] = [2]
filter_conditions

In [ ]:
# 백테스트, 검증, 포트추출

try:
    testing(wd, filter_conditions)
except Exception as e:
    tb = traceback.format_exc()
    print(f"\n\n {tb} \n\n")
    raise e

variables_list = [qqa.variables]
result_df = pd.DataFrame(variables_list)
result_df = result_df.astype("str")

db.engine_upload(upload_df=result_df, table_name="qa_test_results")

In [ ]:
deno_elms = wd.find_elements(By.ID, "커스텀 팩터 1.numerator")
# deno_elms[i].click()
time.sleep(1.5)
# deno = deno_elms[i].get_attribute("value")

In [ ]:
deno_elms[0].get_attribute("value")

In [ ]:
# filter_conditions["custom_factors"]["denominators"] = [1, 2]

In [ ]:
db.engine_upload(upload_df=result_df, table_name="qa_test_results")

In [ ]:
# 백테스트

qqa.__init__()
qqa.test_back_port(wd, kind=0, **filter_conditions)
variables_list = [qqa.variables]
result_df = pd.DataFrame(variables_list)
result_df = result_df.astype("str")
result_df

  
#### 1. 유니버스 선택

In [ ]:
variables_list = []
for i in range(1, 5):
    filter_conditions["universe"] = i
    testing(wd, filter_conditions)
    variables_list.append(qqa.variables)

In [ ]:
result_df = pd.DataFrame(variables_list)
result_df = result_df.astype("str")
result_df

In [ ]:
db.engine_upload(upload_df=result_df, table_name="qa_test_results")
result_df = db.get_tbl("qa_test_results")
result_df

#### 2. 팩터 선택

In [ ]:
# 랜덤 펙터 선택

filter_conditions["factors"] = None
test_counts = 12

variables_list = []
i = 0
while i < test_counts:
    testing(wd, filter_conditions)
    variables_list.append(qqa.variables)
    i += 1

In [ ]:
result_df = pd.DataFrame(variables_list)
result_df = result_df.astype("str")
result_df

db.engine_upload(upload_df=result_df, table_name="qa_test_results")
result_df = db.get_tbl("qa_test_results")
result_df.info()

In [ ]:
# 모든 펙터 선택

filter_conditions["factors"] = "all"
try:
    testing(wd, filter_conditions)
    variables_list = [qqa.variables]
except:
    tb = traceback.format_exc()
    print(f"\n\n** Error **\n {tb} \n\n")

In [ ]:
result_df = pd.DataFrame(variables_list)
result_df = result_df.astype("str")
db.engine_upload(upload_df=result_df, table_name="qa_test_results")

In [ ]:
import traceback
# 모든 팩터 한개씩 선택

from scraper import scraping_factors
all_factors, all_factor_counts = scraping_factors()

# test
# all_factor_counts = 10
fs = []

variables_list = []
i = 11
while i < all_factor_counts:
    # fr = random.randint(0, all_factor_counts-1)
    # filter_conditions["factors"] = [fr]
    filter_conditions["factors"] = [i]
    try:
        testing(wd, filter_conditions)
        variables_list.append(qqa.variables)
    except:
        tb = traceback.format_exc()
        print(f"\n\n** Error **\n {tb} \n\n")
    i += 1

In [ ]:
result_df = pd.DataFrame(variables_list)
result_df = result_df.astype("str")

db.engine_upload(upload_df=result_df, table_name="qa_test_results")
result_df = db.get_tbl("qa_test_results")
result_df.info()

    
#### 3. 제외할 섹터

In [ ]:
filter_conditions["removed_sectors"] = list(range(7, 33))

filter_conditions["universe"] = 1
filter_conditions["removed_filters"] = []
filter_conditions["factors"] = [0]

filter_conditions["conditions"]["stock_counts"] = 100

In [ ]:
qqa.__init__()
    
# 초기 설정 (유니버스, 필터, 팩터)
qqa.test_back_port(wd, kind=1, **filter_conditions)

In [ ]:
# Check sector

sectors = qqa.variables["removed_sectors"]
industries = ast.literal_eval(qqa.variables["industry"])

unremoved = []
for sector in sectors:
    sector = sector.replace(" ", "")
    if sector in industries:
        unremoved.append(sector)
        
if len(unremoved) != 0:
    raise ValueError(f"제외할 섹터 반영이 안되었습니다.\n- Removed sectors: {sectors}\n- Result industries: {industries}\n- Unremoved sectors: {unremoved}")
else:
    print("제외할 섹터 적용이 완료되었습니다.")

In [ ]:
industries

In [ ]:
qqa.variables["portfolio"]

In [ ]:
variables_list = [qqa.variables]
result_df = pd.DataFrame(variables_list)
result_df = result_df.astype("str")
result_df

#### ** 필수 항목 Alert Check **

In [ ]:
filter_conditions["conditions"]

In [ ]:
# 유니버스 선택 안함
filter_conditions["universe"] = 0
testing(wd, filter_conditions)
result_df = pd.DataFrame([qqa.variables])
result_df

In [ ]:
# 팩터 선택 안함
filter_conditions["factors"] = []
testing(wd, filter_conditions)
result_df = pd.DataFrame([qqa.variables])
result_df

In [ ]:
# 백테스트, 트레이딩 설정

variables_list = []
i = 0
while i < len(filter_conditions["conditions"]) - 2:
    filter_conditions["conditions"] = {
                            'init_amounts': 4000,
                            'fees': 1.5,
                            'stock_counts': 12,
                            'stop_loss': 16,
                            'profit': 26,
                            'rebalancing_period': 2,
                            'rebalancing_method': None,
                            'rebalancing_strategy': None,
                        }
    keys = list(filter_conditions["conditions"].keys())
    filter_conditions["conditions"][keys[i]] = None
    
    print(filter_conditions["conditions"])
    
    testing(wd, filter_conditions)
    variables_list.append(qqa.variables)
    
    i += 1

In [ ]:
result_df = pd.DataFrame(variables_list)
result_df = result_df.astype("str")
db.engine_upload(upload_df=result_df, table_name="qa_test_results")

In [ ]:
wd.quit()

---
#### 커스텀 팩터